In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

import requests
from PyPDF2 import PdfMerger
from datetime import datetime
import concurrent.futures
import subprocess

def download_pdf(url, filename):
    try:
        response = requests.get(url)
        if response.status_code == 200 and len(response.content) > 0:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return filename
    except Exception as e:
        print(f"Error downloading {url}: {e}")
    return None

def merge_pdfs(pdf_files, output_filename):
    try:
        merger = PdfMerger()
        for pdf in sorted(pdf_files, key=lambda x: int(x.split('_')[1].split('.')[0])):
            if pdf:
                merger.append(pdf)
        merger.write(output_filename)
        merger.close()
        print(f"Successfully merged {len(pdf_files)} PDFs into {output_filename}")
    except Exception as e:
        print(f"Error merging PDFs: {e}")

def compress_pdf(input_pdf, output_pdf):

    gs_command = [
        "gswin64c", 
        "-sDEVICE=pdfwrite",
        "-dCompatibilityLevel=1.3",
        "-dPDFSETTINGS=/printer",  
        "-dNOPAUSE",
        "-dBATCH",
        "-dDownsampleColorImages=true",
        "-dColorImageResolution=180",
        f"-sOutputFile={output_pdf}",
        input_pdf
    ]
    try:
        subprocess.run(gs_command, check=True)
    except FileNotFoundError:
        print("Ghostscript not found. Please ensure Ghostscript is installed and added to your PATH.")
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")

def download_and_merge_newspaper(date_str):
    base_url = "https://epaper.saamana.com/download.php?file=https://enewspapr.com/News/SAMANA/PUN/{year}/{month}/{day}/{date_str}_{page}.PDF&pageno={page}"
    
    year, month, day = date_str[:4], date_str[4:6], date_str[6:]
    formatted_url = base_url.format(year=year, month=month, day=day, date_str=date_str, page="{page}")

    pdf_files = []
    page = 1

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_page = {}
        while True:
            future = executor.submit(download_pdf, formatted_url.format(page=page), f"page_{page}.pdf")
            future_to_page[future] = page
            page += 1
            if len(future_to_page) >= 10:
                for future in concurrent.futures.as_completed(future_to_page):
                    page_num = future_to_page[future]
                    try:
                        filename = future.result()
                        if filename:
                            pdf_files.append(filename)
                    except Exception as e:
                        print(f"Error downloading page {page_num}: {e}")
                future_to_page.clear()
                if not filename:
                    break
            if page > 40: 
                break

        for future in concurrent.futures.as_completed(future_to_page):
            page_num = future_to_page[future]
            try:
                filename = future.result()
                if filename:
                    pdf_files.append(filename)
            except Exception as e:
                print(f"Error downloading page {page_num}: {e}")

    if pdf_files:
        output_filename = f"UNCOMPRESSED_SAAMANA_PUNE_{date_str}.pdf"
        merge_pdfs(pdf_files, output_filename)

        for pdf in pdf_files:
            os.remove(pdf)

        original_size = os.path.getsize(output_filename) / (1024 * 1024)  
        print(f"Merged {len(pdf_files)} pages into {output_filename} with size {original_size:.2f} MB")

        compressed_output_filename = f"SAAMANA_PUNE_{date_str}.pdf"
        compress_pdf(output_filename, compressed_output_filename)
        
        compressed_size = os.path.getsize(compressed_output_filename) / (1024 * 1024)  
        print(f"Compressed {output_filename} to {compressed_output_filename} with size {compressed_size:.2f} MB")
        
        os.remove(output_filename)

        print(f"Downloaded, merged, and compressed {len(pdf_files)} pages into {compressed_output_filename}")
    else:
        print("No pages were downloaded. Please check the date and try again.")

def send_pdf_to_telegram(pdf_filename, today_date):
    TELEGRAM_BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')
    TELEGRAM_CHAT_ID = os.getenv('TELEGRAM_CHAT_ID')

    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendDocument"
    
    with open(pdf_filename, 'rb') as pdf_file:
        files = {
            'document': pdf_file,
        }
        data = {
            'chat_id': TELEGRAM_CHAT_ID,
            'caption': f"SAAMANA_PUNE_{today_date}.pdf"
        }
        response = requests.post(url, data=data, files=files)

    if response.status_code == 200:
        print("PDF sent to Telegram successfully.")
    else:
        print(f"Failed to send PDF. Response: {response.text}")

if __name__ == "__main__":
    today_date = datetime.now().strftime("%Y%m%d")
    download_and_merge_newspaper(today_date)
    pdf_filename = f"SAAMANA_PUNE_{today_date}.pdf"
    send_pdf_to_telegram(pdf_filename, today_date)